### Notebook used to generate list of congested CA branches, for use in scenario where CA (only) fixes its congested lines.

In [1]:
import numpy as np
import pandas as pd

import os
import sys
sys.path.append("..")

import westernintnet

In [2]:
wecc = westernintnet.WesternIntNet()

Loading sub
Loading bus2sub
Loading bus
Loading genbus
Loading branches
Loading resources
Loading net_generation
Load solar data
Load wind data
Load hydro data
Load demand data
Done loading


In [3]:
CA_zones = [3,4,5,6,7]
CA_buses = wecc.bus.loc[wecc.bus['area'].isin(CA_zones)]
CA_branches = wecc.branches.loc[(wecc.branches['tbus'].isin(CA_buses['busID'])) 
                              & (wecc.branches['fbus'].isin(CA_buses['busID']))]

fix_list = ['california2020Test01','california2030Test01']

In [4]:
datadir = os.path.join('..','data/')
congestion_dfs = {}
for s in fix_list:
    congestion_source = datadir + 'congestion_' + s +'.csv'
    congestion_dfs[s] = pd.read_csv(congestion_source)
    

In [5]:
CA_fix_congestion = {}
for s in fix_list:
    CA_fix_congestion[s] = congestion_dfs[s].loc[(congestion_dfs[s]['line'].isin(CA_branches.index)) 
                                                 & (congestion_dfs[s]['zscore']>=1)]
    filename = datadir + 'congestion_CA_only_' + s + '_zscore_gte_1.csv'
    if not os.path.exists(filename):
        CA_fix_congestion[s].to_csv(filename)
        
    print(s, len(CA_fix_congestion[s]))

california2020Test01 51
california2030Test01 129


#### Extra stats

In [6]:
# How many are transmission lines? Less than half are transmission lines?!?
for s in fix_list:
    print(s)
    print(CA_fix_congestion[s].loc[CA_fix_congestion[s]['dist']==0][['Capacity','hutil>=0p75','hutil>=0p9','hutil1','pvalue']].describe())
    print(CA_fix_congestion[s].loc[CA_fix_congestion[s]['dist']>0][['Capacity','hutil>=0p75','hutil>=0p9','hutil1','dist','pvalue']].describe())
    print()
    

california2020Test01
         Capacity  hutil>=0p75   hutil>=0p9       hutil1  pvalue
count   35.000000    35.000000    35.000000    35.000000    35.0
mean   134.742857  2530.600000  1013.371429   791.657143     0.0
std    149.593241  1969.268873  1417.939310  1227.698180     0.0
min      8.000000   120.000000     0.000000     0.000000     0.0
25%     17.500000  1098.000000     0.000000     0.000000     0.0
50%    103.000000  1790.000000   104.000000     0.000000     0.0
75%    172.500000  3557.000000  3187.000000  2194.500000     0.0
max    594.000000  8774.000000  3216.000000  2894.000000     0.0
          Capacity  hutil>=0p75  hutil>=0p9  hutil1       dist        pvalue
count    16.000000    16.000000   16.000000  16.000  16.000000  1.600000e+01
mean    310.937500   623.125000    3.062500   0.125  18.419250  1.289085e-03
std     246.364897   851.177997    4.479118   0.500  15.486764  5.126887e-03
min     191.000000    41.000000    0.000000   0.000   0.846000  0.000000e+00
25%     2

In [10]:
CA_fix_congestion['california2030Test01'].loc[CA_fix_congestion['california2030Test01']['dist']==0].head()

,line,Capacity,hutil1,hutil0p9-1,hutil0p8-0p9,hutil0p75-0p8,hutil>=0p9,hutil>=0p8,hutil>=0p75,dist,zscore,pvalue
2707,2707,95,199,909,547,262,1108,1655,1917,0.0,230.463831,0.000000e+00
2708,2708,59,0,0,12,111,0,12,123,0.0,7.210948,2.777778e-13
2763,2763,70,2,133,750,417,135,885,1302,0.0,153.930652,0.000000e+00
2764,2764,29,386,869,491,213,1255,1746,1959,0.0,235.690487,0.000000e+00
2771,2771,70,22,289,860,319,311,1171,1490,0.0,177.326160,0.000000e+00


In [11]:
wecc.branches.loc[2707]

fbus            20305
tbus            20306
r            0.005847
x            0.159874
b                   0
rateA           95.46
rateB               0
rateC               0
ratio               1
angle               0
status              1
angmin              0
angmax              0
Pf             -43.97
Qf               2.98
Pt              44.08
Qt                  0
mu_Sf               0
mu_St               0
mu_angmin           0
mu_angmax     0.0000;
from_lat      38.8348
from_lon     -122.768
to_lat        38.8348
to_lon       -122.768
Name: 2707, dtype: object